Inspired by IRONSIGHT(@ravi02516)'s notebook ["End to End EffiecientDet Training Keras"](https://www.kaggle.com/ravi02516/end-to-end-effiecientdet-training-keras).

Thanks😉

In [ ]:
!git clone https://github.com/google/automl.git

In [ ]:
!pip install pycocotools

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import pickle
os.chdir("/kaggle/working/automl/efficientdet")

SEED = 777
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# latest commit on 7/29/2020
!git checkout e2ca3361f4c00380a7002e2c9a7ebda1b7f06787

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    ds_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    ds_strategy = tf.distribute.get_strategy()

print("REPLICAS: ", ds_strategy.num_replicas_in_sync)

In [ ]:
TF_REC_DS_PATH = KaggleDatasets().get_gcs_path('global-wheat-detection-tfrecord')
training_file_pattern = [TF_REC_DS_PATH + f'/{fold}.tfrecord' for fold in range(0,9)]
validation_file_pattern = TF_REC_DS_PATH + f'/{9}.tfrecord'

In [ ]:
%%time
import os
from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
from tensorflow.python.client import device_lib
import dataloader
import hparams_config
import utils
from keras import train_lib

FLAGS={'tpu':None,'gcp_project':None,'tpu_zone':None,'eval_master':'',
      'eval_name':None,'strategy':None,'num_cores':8,'use_fake_data':False,
      'use_xla':False,'model_dir':"/model",'hparams':'num_classes=1, max_instances_per_image=120, clip_gradients_norm=2.0','batch_size':64,
      'eval_samples':342,'iterations_per_loop':100,'training_file_pattern':training_file_pattern,
      'validation_file_pattern':validation_file_pattern,'val_json_file':None,'testdev_dir':None,
      'num_examples_per_epoch':3079,'num_epochs':None,'mode':'train',
      'model_name':'efficientdet-d0','eval_after_training':False,
      'debug':False,'profile':False,'min_eval_interval':180,
      'eval_timeout':None}


if tpu:
    FLAGS['strategy'] = 'tpu'

# Parse and override hparams
config = hparams_config.get_detection_config(FLAGS["model_name"])
config.override(FLAGS["hparams"])

# Parse image size in case it is in string format.
config.image_size = utils.parse_image_size(config.image_size)


# Check data path
if FLAGS["mode"] in ('train',
                    'train_and_eval') and FLAGS["training_file_pattern"] is None:
    raise RuntimeError('You must specify --training_file_pattern for training.')
if FLAGS["mode"] in ('eval', 'train_and_eval'):
    if FLAGS["validation_file_pattern"] is None:
        raise RuntimeError('You must specify --validation_file_pattern '
                        'for evaluation.')


steps_per_epoch = FLAGS['num_examples_per_epoch'] // FLAGS['batch_size']
validation_steps = FLAGS['eval_samples'] // FLAGS['batch_size']

params = dict(
    config.as_dict(),
    model_name=FLAGS["model_name"],
    iterations_per_loop=FLAGS["iterations_per_loop"],
    model_dir=FLAGS["model_dir"],
    num_examples_per_epoch=FLAGS["num_examples_per_epoch"],
    steps_per_epoch=steps_per_epoch,
    strategy=FLAGS["strategy"],
    batch_size=FLAGS["batch_size"] // ds_strategy.num_replicas_in_sync,
    num_shards=ds_strategy.num_replicas_in_sync,
    val_json_file=FLAGS["val_json_file"],
    testdev_dir=FLAGS["testdev_dir"],
    mode=FLAGS["mode"],
    iou_loss_type='ciou',
    moving_average_decay=0
)

# set mixed precision policy by keras api.
precision = utils.get_precision(params['strategy'], params['mixed_precision'])
policy = tf.keras.mixed_precision.experimental.Policy(precision)
tf.keras.mixed_precision.experimental.set_policy(policy)

def get_dataset(is_training, params):
    file_pattern = (
        FLAGS["training_file_pattern"]
        if is_training else FLAGS["validation_file_pattern"])
    return dataloader.InputReader(
        file_pattern,
        is_training=is_training,
        use_fake_data=FLAGS["use_fake_data"],
        max_instances_per_image=config.max_instances_per_image)(
            params)
        
tf.keras.backend.clear_session()
with ds_strategy.scope():
    model = train_lib.EfficientDetNetTrain(params['var_freeze_expr'], params['model_name'], config)
    height, width = utils.parse_image_size(params['image_size'])
    model.build((params['batch_size'], height, width, 3))
    model.summary()

    model.compile(
        optimizer=train_lib.get_optimizer(params),
        loss={
            'box_loss':
                train_lib.BoxLoss(
                    params['delta'],reduction=tf.keras.losses.Reduction.NONE),
            'box_iou_loss':
                train_lib.BoxIouLoss(
                    params['iou_loss_type'],
                    params['min_level'],
                    params['max_level'],
                    params['num_scales'],
                    params['aspect_ratios'],
                    params['anchor_scale'],
                    params['image_size'],
                    reduction=tf.keras.losses.Reduction.NONE),
            'class_loss':
                train_lib.FocalLoss(
                    params['alpha'],
                    params['gamma'],
                    label_smoothing=params['label_smoothing'],
                    reduction=tf.keras.losses.Reduction.NONE)
        })


history = model.fit(
    get_dataset(True, params=params),
    steps_per_epoch=steps_per_epoch,
    epochs=15,
    validation_data=get_dataset(False, params=params),
    validation_steps=validation_steps
)

In [ ]:
# reference: https://qiita.com/mgmk2/items/556ca3f4471ada9c69f0
def get_model_weights_as_numpy(model):
    weights = {}
    for v in model.weights:
        weights[v.name] = v.numpy()
    return weights

def set_model_weights_from_numpy(weights, model):
    for v in model.weights:
        if v.name in weights.keys():
            v.assign(weights[v.name])
        else:
            print('Not loaded weights: ' + v.name)

model_weights = get_model_weights_as_numpy(model)

In [ ]:
%%time
from keras import efficientdet_keras
from object_detection import tf_example_decoder
import inference
from PIL import Image
import glob

imgs = []

for f in glob.iglob('/kaggle/input/global-wheat-detection/test/*'):
    imgs.append(np.array(Image.open(f)))

    
config = hparams_config.get_efficientdet_config(params['model_name'])
config.override(FLAGS["hparams"])
config.is_training_bn = False
config.image_size = params['image_size']
config.nms_configs.score_thresh = 0.2

tf.keras.backend.clear_session()
with ds_strategy.scope():
    model_inference = efficientdet_keras.EfficientDetModel(config=config)
    model_inference.build((1, None, None, 3))
    set_model_weights_from_numpy(model_weights, model_inference)

boxes, scores, classes, valid_len = model_inference(imgs, training=False, post_mode='global')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(30,50))

for i, img in enumerate(imgs):
    length = valid_len[i]
    img = inference.visualize_image(
        img,
        boxes[i].numpy()[:length],
        classes[i].numpy().astype(np.int)[:length],
        scores[i].numpy()[:length],
        min_score_thresh=config.nms_configs.score_thresh,
        max_boxes_to_draw=config.nms_configs.max_output_size)
    fig.add_subplot(5, 2, i+1)
    plt.imshow(img)

fig.tight_layout()
plt.show()

In [ ]:
import shutil
shutil.rmtree("/kaggle/working/automl")